In [ ]:

missing_buyer_addr = predictions_df[predictions_df['BUYER ADDRESS'].isna()].index.tolist()
print(len(missing_buyer_addr))

state = r"\b(?:alabama|al|alaska|ak|arizona|az|arkansas|ar|california|ca|colorado|co|connecticut|ct|delaware|de|florida|fl|georgia|ga|hawaii|hi|idaho|id|illinois|il|indiana|in|iowa|ia|kansas|ks|kentucky|ky|louisiana|la|maine|me|maryland|md|massachusetts|ma|michigan|mi|minnesota|mn|mississippi|ms|missouri|mo|montana|mt|nebraska|ne|nevada|nv|new hampshire|nh|new jersey|nj|new mexico|nm|new york|ny|north carolina|nc|north dakota|nd|ohio|oh|oklahoma|ok|oregon|or|pennsylvania|pa|rhode island|ri|south carolina|sc|south dakota|sd|tennessee|tn|texas|tx|utah|ut|vermont|vt|virginia|va|washington|wa|west virginia|wv|wisconsin|wi|wyoming|wy)\b"
zip_code = r"\b(?:\d{5}(?:\s*-\s*\d{1,4})?)\b"

def build_name_regexes(buyer_name_raw):

    names = re.split(r'\s*,\s*|\s+and\s+', buyer_name_raw, flags=re.IGNORECASE)
    regex_patterns = []

    for name in names:
        name = name.strip()

        name_parts = name.split()

        if not name_parts:
            continue

        pattern = re.escape(name_parts[0])

        for middle in name_parts[1:-1]:
            pattern += r'\s+(?:' + re.escape(middle) + r'|' + re.escape(middle[0]) + r'\.?)?'

        if len(name_parts) > 1:
            pattern += r'\s+' + re.escape(name_parts[-1])

        regex_patterns.append(pattern)

    return regex_patterns



recovered = 0
for i in missing_buyer_addr:


  file_name = predictions_df.loc[i, 'IMAGENAME']
  print(file_name)
  folder_path = '/content/drive/MyDrive/ocr_output_2000/'
  full_path = f"{folder_path}{file_name}"

  with open(full_path, 'r', encoding='utf-8') as f:
    doc_text = f.read()


  buyer_name = predictions_df.loc[i, 'BUYER NAME']
  buyer_org = predictions_df.loc[i, 'BUYER ORG']

  if pd.notna(buyer_name):

    buyer_name = re.sub(r'\s+([.,])', r'\1', buyer_name)
    buyer_name_patterns = build_name_regexes(buyer_name)

  elif pd.notna(buyer_org):
    buyer_org = re.sub(r'\s+([.,])', r'\1', buyer_org)
    buyer_name_patterns = [rf"{buyer_org}"]

  else:
    continue

  buyer_addrs = []
  for buyer_name_pattern in buyer_name_patterns:

    buyer_addr_pattern = rf"{buyer_name_pattern}.{{0,100}}{state}\s+{zip_code}"
    buyer_addr = re.search(buyer_addr_pattern, doc_text, re.IGNORECASE | re.DOTALL)

    if buyer_addr is not None:
      buyer_addrs.append(buyer_addr.group())

  if len(buyer_addrs) > 0:
    predictions_df.at[i, 'BUYER ADDRESS'] = ' '.join(buyer_addrs)
    recovered += 1
    print(buyer_addrs)
    print("\n")

print(recovered)
str_columns = ["APN","consideration","recording_date","doc_date","signature_date","effective_date","book_num","page_num","document_number"]
predictions_df[str_columns] = predictions_df[str_columns].astype(str)
predictions_df.to_excel("2000_docs_predictions.xlsx", index=False, engine="openpyxl")




In [ ]:
def all_matches(pattern, text, address_pattern):
  text = text.lower()
  matches = re.finditer(pattern, text)
  for m in matches:
    start, end = m.span()
    expanded_text = text[start-50: end+50]
    full_addr = re.search(address_pattern, expanded_text, re.DOTALL)
    if full_addr is not None:
      return full_addr.group()

  return None


def normalize_address(combined_df, col):

  street_number = r"(?:\b\d{1,6}\s+)"
  middle_address = r"(?:.){0,75}"
  state = r"\b(?:alabama|a\.?l\.?|alaska|a\.?k\.?|arizona|a\.?z\.?|arkansas|a\.?r\.?|california|c\.?a\.?|colorado|c\.?o\.?|connecticut|c\.?t\.?|delaware|d\.?e\.?|florida|f\.?l\.?|georgia|g\.?a\.?|hawaii|h\.?i\.?|idaho|i\.?d\.?|illinois|i\.?l\.?|indiana|i\.?n\.?|iowa|i\.?a\.?|kansas|k\.?s\.?|kentucky|k\.?y\.?|louisiana|l\.?a\.?|maine|m\.?e\.?|maryland|m\.?d\.?|massachusetts|m\.?a\.?|michigan|m\.?i\.?|minnesota|m\.?n\.?|mississippi|m\.?s\.?|missouri|m\.?o\.?|montana|m\.?t\.?|nebraska|n\.?e\.?|nevada|n\.?v\.?|new hampshire|n\.?h\.?|new jersey|n\.?j\.?|new mexico|n\.?m\.?|new york|n\.?y\.?|north carolina|n\.?c\.?|north dakota|n\.?d\.?|ohio|o\.?h\.?|oklahoma|o\.?k\.?|oregon|o\.?r\.?|pennsylvania|p\.?a\.?|rhode island|r\.?i\.?|south carolina|s\.?c\.?|south dakota|s\.?d\.?|tennessee|t\.?n\.?|texas|t\.?x\.?|utah|u\.?t\.?|vermont|v\.?t\.?|virginia|v\.?a\.?|washington|w\.?a\.?|west virginia|w\.?v\.?|wisconsin|w\.?i\.?|wyoming|w\.?y\.?)\b"
  zip_code = r"(?:\d{5}(?:\s*-\s*\d{1,4})?)\b"

  address_pattern = rf"{street_number}{middle_address}{state}\s*(?:,)?\s*(?:{zip_code})?"

  folder_path = "/content/drive/MyDrive/ocr_output_2000/"

  for i in range(len(combined_df)):

    filename = combined_df.loc[i, 'IMAGENAME'] + ".txt"
    file_path = os.path.join(folder_path, filename)
    with open(file_path, 'r', encoding='utf-8') as file:
      text = file.read()

    text = re.sub(r'\s*([,.:#\-])\s*', r'\1', text)
    addr = combined_df.loc[i, col]
    if pd.notna(addr):

      addr = re.sub(r'\s*([,.:#\-])\s*', r'\1', addr)
      addr = addr.replace("##", "")
      pattern = re.escape(addr)
      m = re.search(pattern, text.lower())

      if m:

        start, end = m.span()
        full_addr = all_matches(pattern, text, address_pattern)
        combined_df.loc[i, col] = full_addr

      else:
        combined_df.loc[i, col] = addr


  return combined_df

col = "BUYER ADDRESS"
combined_df = normalize_address(combined_df, col)
print(combined_df[col])
